In [2]:
import os,glob
import csv

class_to_num={}

class_name_list=os.listdir('C:/Users/beichen/dataset/pork')
print(class_name_list)

for class_name in class_name_list:  
    class_to_num[class_name]=len(class_to_num.keys())

print(class_to_num)  #{'class1': 0, 'class2': 1, 'class3': 2}

image_dir=[]
for class_name in class_name_list:
    image_dir+=glob.glob(os.path.join('C:/Users/beichen/dataset/pork',class_name,'*.jpg'))
print(image_dir)

import random 
random.shuffle(image_dir)

with open('myself_date.csv', mode='w', newline='') as f:
    writer=csv.writer(f)
    for image in image_dir:
        class_name=image.split(os.sep)[-2]
        label=class_to_num[class_name]
        writer.writerow([image,label])
print('finished')

['class1', 'class2', 'data.txt']
{'class1': 0, 'class2': 1, 'data.txt': 2}
['C:/Users/beichen/dataset/pork\\class1\\06e8d71d0dee20e2ec6462ad304385bc.jpg', 'C:/Users/beichen/dataset/pork\\class1\\314a43714111772cac59f3eaf6b7dbdb.jpg', 'C:/Users/beichen/dataset/pork\\class1\\5776dfad18c42a51d5d61dd036989a86.jpg', 'C:/Users/beichen/dataset/pork\\class1\\75b40c09260636aefd5a4c7230c4f92a.jpg', 'C:/Users/beichen/dataset/pork\\class1\\8c26dc0b6a21ed31faef779af6dd289e.jpg', 'C:/Users/beichen/dataset/pork\\class1\\8c7253e2001fd5c23ed98afcc8fbbab9.jpg', 'C:/Users/beichen/dataset/pork\\class1\\d45d34cc1687a30e9cdae8af81d73afc.jpg', 'C:/Users/beichen/dataset/pork\\class1\\IMG_20230327_143302.jpg', 'C:/Users/beichen/dataset/pork\\class1\\IMG_20230327_143307.jpg', 'C:/Users/beichen/dataset/pork\\class1\\IMG_20230327_143324.jpg', 'C:/Users/beichen/dataset/pork\\class1\\IMG_20230327_143327.jpg', 'C:/Users/beichen/dataset/pork\\class1\\IMG_20230327_143332.jpg', 'C:/Users/beichen/dataset/pork\\class1\\I

In [3]:
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import csv
from PIL import Image

class   makedataset(Dataset):
    def __init__(self,csv_filename,resize,mode):
        super(makedataset,self).__init__()

        self.csv_filename=csv_filename
        self.resize=resize
        self.image,self.label=self.load_csv()

        if mode=='train':   #80%训练
            self.image=self.image[:int(0.8*len(self.image))]
            self.label=self.label[:int(0.8*len(self.label))]
        elif mode=='val':  #10%
            self.image = self.image[:int(0.1*len(self.image))]
            self.label = self.label[:int(0.1*len(self.label))]
        else:   #10%
            self.image = self.image[:int(0.1 * len(self.image)):]
            self.label = self.label[:int(0.1 * len(self.label)):]
    def load_csv(self):
        image,label=[],[]
        with open(self.csv_filename) as f:
            reader=csv.reader(f)
            for row in reader:
                i,l=row
                image.append(i)
                label.append(int(l))
        return image,label

    def __len__(self):
        return  len(self.image)

    def __getitem__(self, idx):
        tf=transforms.Compose([lambda x:Image.open(x).convert('RGB'),
                               transforms.Resize((32,32)),
                               transforms.ToTensor()])
        image_tensor=tf(self.image[idx])
        label_tensor=torch.tensor(self.label[idx])
        return image_tensor,label_tensor
train_db=makedataset('myself_date.csv', 32, 'train')
from  torch.utils.data import DataLoader
train_loder=DataLoader(train_db,batch_size=30)
i=0
for x,y in train_loder:
    print(x.shape)
    print(y.shape)
    i=i+1
print(i)

torch.Size([30, 3, 32, 32])
torch.Size([30])
torch.Size([30, 3, 32, 32])
torch.Size([30])
torch.Size([30, 3, 32, 32])
torch.Size([30])
torch.Size([30, 3, 32, 32])
torch.Size([30])
torch.Size([30, 3, 32, 32])
torch.Size([30])
torch.Size([30, 3, 32, 32])
torch.Size([30])
torch.Size([30, 3, 32, 32])
torch.Size([30])
torch.Size([30, 3, 32, 32])
torch.Size([30])
torch.Size([30, 3, 32, 32])
torch.Size([30])
torch.Size([30, 3, 32, 32])
torch.Size([30])
torch.Size([30, 3, 32, 32])
torch.Size([30])
torch.Size([30, 3, 32, 32])
torch.Size([30])
torch.Size([30, 3, 32, 32])
torch.Size([30])
torch.Size([30, 3, 32, 32])
torch.Size([30])
torch.Size([30, 3, 32, 32])
torch.Size([30])
torch.Size([30, 3, 32, 32])
torch.Size([30])
torch.Size([30, 3, 32, 32])
torch.Size([30])
torch.Size([30, 3, 32, 32])
torch.Size([30])
torch.Size([30, 3, 32, 32])
torch.Size([30])
torch.Size([4, 3, 32, 32])
torch.Size([4])
20


In [4]:
import torch
import torch.nn as nn
from einops.layers.torch import Rearrange


class mymodel(nn.Module):
    def __init__(self,image_shape):
        super(mymodel,self).__init__()

        s=image_shape[1]
        self.Down_sample=nn.MaxPool2d(kernel_size=2,stride=2)
        self.Active_f=nn.ReLU()

        self.Conv1=nn.Conv2d(3,16,3,1,padding=1)
        self.BN1=nn.BatchNorm2d(16)

        self.Conv2 = nn.Conv2d(16, 16, 3, 1, padding=1)
        self.BN2 = nn.BatchNorm2d(16)

        self.FC=nn.Sequential(Rearrange('b c h w -> b (c h w)'),

                              nn.Linear(16*(s//4)**2,16),
                              nn.Dropout(0.5),
                              nn.ReLU(),

                              nn.Linear(16,3))


    def forward(self,input):
        x = self.Active_f(self.Down_sample(self.BN1(self.Conv1(input))))
        x = self.Active_f(self.Down_sample(self.BN2(self.Conv2(x))))
        output=self.FC(x)
        return output
model = mymodel([3,128,128])
img=torch.randn(1,3,128,128)
print(model(img).shape)


torch.Size([1, 3])


In [10]:
import torch
from torch import optim
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader

#from MakeDataset import makedataset
#from MyModel import mymodel
   ########################
train_db = makedataset('myself_date.csv',32,mode='train')
val_db = makedataset('myself_date.csv',32,mode='val')
test_db = makedataset('myself_date.csv',32,mode='test')
train_loader = DataLoader(train_db,batch_size=4,shuffle=True)
val_loader = DataLoader(val_db,batch_size=len(val_db))
test_loader = DataLoader(test_db,batch_size=len(test_db))
print('num_train',len(train_loader.dataset))
print('num_val',len(val_loader.dataset))
print('num_test',len(test_loader.dataset))

#在cuda上运行
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

def compute_acc(model,loader):
    model.eval()

    correct = 0
    total = len(loader.dataset)
    for x,y in loader:
        with torch.no_grad():
            logits = model(x)
            pred = logits.argmax(dim=1)
        correct += torch.eq(pred,y).sum().float().item()
    return correct / total

model = mymodel([3,32,32])
optimizer = optim.Adam(model.parameters(),lr=1e-3)
loss_f = CrossEntropyLoss()

def train():
    best_val_acc,best_epoch = 0,0
    for epoch in range(5):
        for x, y in train_loader:
            model.train()
            pred = model(x)
            loss = loss_f(pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        val_acc = compute_acc(model, val_loader)
        print('Epoch:', epoch, 'Loss:', loss, 'Val_acc:', val_acc)
        if val_acc > best_val_acc:
            torch.save(model.state_dict(),'weights_mymodel.pt')
            best_val_acc = val_acc
            best_epoch = epoch
    print('Best_Val_acc:',best_val_acc,'Best_epoch:',best_epoch)

def test():
    model.load_state_dict(torch.load('weights_mymodel.pt'))
    print('Test_acc:',compute_acc(model,test_loader))

train()
test()


num_train 574
num_val 71
num_test 71
cuda:0
Epoch: 0 Loss: tensor(0.5882, grad_fn=<NllLossBackward0>) Val_acc: 0.8450704225352113
Epoch: 1 Loss: tensor(0.4427, grad_fn=<NllLossBackward0>) Val_acc: 0.7464788732394366
Epoch: 2 Loss: tensor(0.8132, grad_fn=<NllLossBackward0>) Val_acc: 0.8169014084507042
Epoch: 3 Loss: tensor(1.0865, grad_fn=<NllLossBackward0>) Val_acc: 0.8591549295774648
Epoch: 4 Loss: tensor(0.9385, grad_fn=<NllLossBackward0>) Val_acc: 0.8450704225352113
Best_Val_acc: 0.8591549295774648 Best_epoch: 3
Test_acc: 0.8591549295774648


In [9]:
import matplotlib.pyplot as plt
import numpy as np

activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook
model.self.conv1.register_forward_hook(get_activation('self.Conv1'))

# 在0维上增加维度
# data.unsqueeze_(0)
for i,(data,target) in enumerate(test_loader):
  if i>=1:
    break
  print (data.shape)
  output = resnet18(data.to(device))

act = activation['layer1_conv1']
plt.imshow(np.transpose(data[1],(1,2,0)).detach().cpu().numpy())
plt.show()
plt.figure(figsize=(8*2,8*2))
cnt = 0
for j in range(act.size()[1]):
  cnt = cnt + 1
  plt.subplot(np.floor(np.sqrt(act.size()[1])),np.floor(np.sqrt(act.size()[1])),cnt)
  plt.imshow(act[1][cnt-1].detach().cpu().numpy())
plt.show()


AttributeError: 'mymodel' object has no attribute 'self'

In [46]:
from PIL import Image,ImageFont,ImageDraw
import torch
#from MyModel import mymodel
import PIL
import cv2
from torchvision import transforms
import numpy as np

#image = Image.open(r"C:/Users/beichen/dataset/pork/class2/IMG_2472.jpg")
#image.show()

class_names = ['0','1']

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = mymodel([3,32,32])
model.load_state_dict(torch.load("weights_mymodel.pt"))
model.to(device)
model.eval()
print(model)
img_path = "C:/Users/beichen/dataset/pork/class1/IMG_20230327_143335.jpg"

transform_valid = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
  ]
)
img = Image.open(img_path)
img_ = transform_valid(img).unsqueeze(0) #拓展维度

img_ = img_.to(device)
outputs = model(img_)

_, indices = torch.max(outputs,1)
percentage = torch.nn.functional.softmax(outputs, dim=1)[0] * 100
perc = percentage[int(indices)].item()
result = class_names[indices]
print('预测类别为:', result,'(0代表五花肉，1代表梅花肉)')

#得到预测结果，并且从大到小排序
_, indices = torch.sort(outputs, descending=True)
#打印每个预测值的百分数
percentage = torch.nn.functional.softmax(outputs, dim=1)[0] * 100
print([(class_names[idx], percentage[idx].item()) for idx in indices[0][:2]])

# bk_img = cv2.imread("C:/Users/.../file.jpg")
# #设置需要显示的字体
# fontpath = "font/simsun.ttc"
# font = ImageFont.truetype(fontpath, 32)
# img_pil = Image.fromarray(bk_img)
# draw = ImageDraw.Draw(img_pil)
# draw.text((20, 30),  "这张图片的类别可能是："+str(result), font = font, fill = (25, 25, 25))
# bk_img = np.array(img_pil)
# cv2.imshow("add_text",bk_img)
# cv2.waitKey()
# cv2.imwrite("add_text.jpg",bk_img)


# 创建一个照片文件
image = Image.open(img_path)
fontpath = "font/simsun.ttc"
font = ImageFont.truetype(fontpath, 50)
draw = ImageDraw.Draw(image)



text = "这张图片中肉的类别可能是："+str(result)+"\n<0代表五花肉\n1代表梅花肉>\n"

# drawing text size
draw.text((50, 50), text, font=font,fill='yellow', align="left")

image.show()


mymodel(
  (Down_sample): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Active_f): ReLU()
  (Conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (BN1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (Conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (BN2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (FC): Sequential(
    (0): Rearrange('b c h w -> b (c h w)')
    (1): Linear(in_features=1024, out_features=16, bias=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=3, bias=True)
  )
)
预测类别为: 0 (0代表五花肉，1代表梅花肉)
[('0', 81.33589172363281), ('1', 18.66387939453125)]
